In [4]:

import numpy as np
import itertools
from typing import List
from core_components.maps.base import ascii_graphic
from core_components.maps.library import DEFAULT_MANIFEST

manifest = DEFAULT_MANIFEST

def create_states(n: int) -> List[str]:
    """
    Generates a list of all possible binary strings for a given bit length n.

    Args:
        n: The desired number of bits (length of the binary strings).

    Returns:
        A list of strings, where each string is a unique n-bit binary representation.
    """
    if n < 0:
        raise ValueError("Bit length must be a non-negative integer.")
        
    # itertools.product('01', repeat=n) generates all combinations of '0' and '1' 
    # of length n as tuples, which are then joined into strings.
    return [''.join(i) for i in itertools.product('01', repeat=n)]

def graphics_state_assignment(manifest):
    statebits = manifest['statebits']
    n = len(statebits)
    manifest['states'] = create_states(n)

    graphics = manifest['graphics']
    states = manifest['states']
    labels = manifest['color_palette']['labels']

    # Assign allowable states to graphics based on their fixed state bits
    for graphic in graphics.values():

        # Screen through each state and assign it to a graphic if it matches the fixed state bits
        for state in states:
            add_state = False

            for idx, bit in enumerate(state):
                fixed_bit = graphic['fixed_state_bits'][statebits[idx]]
                if fixed_bit is None:
                    pass

                elif fixed_bit == int(bit):
                    add_state = True

                else:
                    add_state = False
                    break

            if add_state:
                # Screen through each label and assign a color state label to the state
                for label_name, label_bits in labels.items():
                    use_label = False
                    for idx, bit in enumerate(state):
                        if label_bits[statebits[idx]] is None:
                            use_label = True
                        
                        elif label_bits[statebits[idx]] == int(bit):
                            use_label = True

                        else:
                             # Placeholder for actual color name
                             use_label = False
                             break
                        
                    if use_label:
                        graphic['state_labels'][state] = label_name

    return manifest

In [5]:
manifest

{'dimensions': {'grid_size': ([80], [50])},
 'state': {'bits': ('blocks_movement', 'blocks_vision', 'visible', 'seen'),
  'names': ('0000',
   '0001',
   '0010',
   '0011',
   '0100',
   '0101',
   '0110',
   '0111',
   '1000',
   '1001',
   '1010',
   '1011',
   '1100',
   '1101',
   '1110',
   '1111'),
  'tuples': ((0, 0, 0, 0),
   (0, 0, 0, 1),
   (0, 0, 1, 0),
   (0, 0, 1, 1),
   (0, 1, 0, 0),
   (0, 1, 0, 1),
   (0, 1, 1, 0),
   (0, 1, 1, 1),
   (1, 0, 0, 0),
   (1, 0, 0, 1),
   (1, 0, 1, 0),
   (1, 0, 1, 1),
   (1, 1, 0, 0),
   (1, 1, 0, 1),
   (1, 1, 1, 0),
   (1, 1, 1, 1)),
  'dtype_labels': {'fixed_bits': {'shroud': {'blocks_movement': None,
     'blocks_vision': None,
     'visible': 0,
     'seen': 0},
    'visible': {'blocks_movement': None,
     'blocks_vision': None,
     'visible': 1,
     'seen': 1},
    'first_look': {'blocks_movement': None,
     'blocks_vision': None,
     'visible': 1,
     'seen': 0},
    'explored': {'blocks_movement': None,
     'blocks_vision': 

In [154]:
# graphics = manifest['graphics']
# states = manifest['state_definition']['names']
# labels = manifest['state_definition']['dtype_labels']

# # Assign allowable states to graphics based on their fixed state bits
# for graphic in graphics.values():

def _intialize_state_map(x) -> None:
    state_label_map = []
    statebits = x['state']['bits']
    dtype_label_bits = x['state']['dtype_labels']['fixed_bits']

    for dtype_label, state_def in dtype_label_bits.items():        
        for idx, state in enumerate(x['state']['tuples']):
                # Screen through each state and assign it to a graphic if it matches the fixed state bits
                label = None
                add_state = True
                for idx, bit in enumerate(state):
                    fixed_bit = state_def[statebits[idx]]
                    if fixed_bit is None:
                        pass

                    elif fixed_bit == int(bit):
                        label = dtype_label

                    else:
                        add_state = False
                        break
                
                if add_state:
                    state_label_map.append(label)

    x['state']['state_label_map'] = tuple(state_label_map)
_intialize_state_map(manifest)

In [155]:
manifest

{'dimensions': {'grid_size': ([80], [50])},
 'state': {'bits': ('blocks_movement', 'blocks_vision', 'visible', 'seen'),
  'names': ('0000',
   '0001',
   '0010',
   '0011',
   '0100',
   '0101',
   '0110',
   '0111',
   '1000',
   '1001',
   '1010',
   '1011',
   '1100',
   '1101',
   '1110',
   '1111'),
  'tuples': ((0, 0, 0, 0),
   (0, 0, 0, 1),
   (0, 0, 1, 0),
   (0, 0, 1, 1),
   (0, 1, 0, 0),
   (0, 1, 0, 1),
   (0, 1, 1, 0),
   (0, 1, 1, 1),
   (1, 0, 0, 0),
   (1, 0, 0, 1),
   (1, 0, 1, 0),
   (1, 0, 1, 1),
   (1, 1, 0, 0),
   (1, 1, 0, 1),
   (1, 1, 1, 0),
   (1, 1, 1, 1)),
  'dtype_labels': {'fixed_bits': {'shroud': {'blocks_movement': None,
     'blocks_vision': None,
     'visible': 0,
     'seen': 0},
    'visible': {'blocks_movement': None,
     'blocks_vision': None,
     'visible': 1,
     'seen': 1},
    'first_look': {'blocks_movement': None,
     'blocks_vision': None,
     'visible': 1,
     'seen': 0},
    'explored': {'blocks_movement': None,
     'blocks_vision': 

In [209]:
n_state_bits = 4
grid_shape = [10, 10]
transposition = [1, 2, 0]
n_dims = 4 # 2D Grid, 1D State Vector, 1D State Bit

states = np.array(tuple([i for i in itertools.product([0,1], repeat=n_state_bits)])) # 1D State Vector, 1D State Bit
states = np.concatenate([states] * grid_shape[0]).reshape(grid_shape[0], 2 ** n_state_bits, n_state_bits) # +1D Grid
states = np.stack([states] * grid_shape[1], axis = 1) # +1D Grid

grid = [np.full([*grid_shape], np.random.randint(2), dtype=int) for _ in range(n_state_bits)] # 1D State Bit x 2D Grids

grid = np.stack(grid, axis = 0).transpose(*transposition).reshape(*grid_shape, n_state_bits) # 1D State Bit, 2D Grids
grid[0:2,0:2] = np.random.randint(2, size=[2,2,4]) # 1D State Bit, 2D Grids
grid = np.stack([grid] * 2 ** n_state_bits, axis=n_dims - 2) # + 1D State Vector

x = np.where(np.all(grid == states, axis=n_dims - 1))

state_index_map = np.full(grid_shape, fill_value=-1, dtype=int)
state_index_map[x[0], x[1]] = x[2]
state_index_map.shape


state_space_label_map = manifest['state']['state_label_map']
label = state_space_label_map[np.unique(state_index_map)[0]]
print(np.unique(state_index_map),label)

[ 7 10 13 15] visible


In [212]:
for index in np.unique(state_index_map):
    print(state_space_label_map[index])
    print(state_index_map == index)

visible
[[ True False False False False False False False False False]
 [False False False False False False False False False False]
 [False False False False False False False False False False]
 [False False False False False False False False False False]
 [False False False False False False False False False False]
 [False False False False False False False False False False]
 [False False False False False False False False False False]
 [False False False False False False False False False False]
 [False False False False False False False False False False]
 [False False False False False False False False False False]]
first_look
[[False  True False False False False False False False False]
 [False False False False False False False False False False]
 [False False False False False False False False False False]
 [False False False False False False False False False False]
 [False False False False False False False False False False]
 [False False False False False Fal

In [108]:
DEFAULT_MANIFEST = graphics_state_assignment(DEFAULT_MANIFEST)
DEFAULT_MANIFEST

{'dimensions': {'grid_size': ([80], [50])},
 'statebits': ('blocks_movement', 'blocks_vision', 'visible', 'seen'),
 'states': ['0000',
  '0001',
  '0010',
  '0011',
  '0100',
  '0101',
  '0110',
  '0111',
  '1000',
  '1001',
  '1010',
  '1011',
  '1100',
  '1101',
  '1110',
  '1111'],
 'dtypes': {'tile_state': None,
  'tile_color': None,
  'tile_graphic': None,
  'tile_grid': None},
 'color_palette': {'labels': {'shroud': {'blocks_movement': None,
    'blocks_vision': None,
    'visible': 0,
    'seen': 0},
   'visible': {'blocks_movement': None,
    'blocks_vision': None,
    'visible': 1,
    'seen': 1},
   'first_look': {'blocks_movement': None,
    'blocks_vision': None,
    'visible': 1,
    'seen': 0},
   'explored': {'blocks_movement': None,
    'blocks_vision': None,
    'visible': 0,
    'seen': 1}},
  'colors': {'fill_bluebell': (32, (255, 255, 255), (50, 50, 150)),
   'fill_light_yellow': (32, (255, 255, 255), (200, 180, 50)),
   'fill_black': (32, (255, 255, 255), (0, 0, 0)

In [25]:
dict.fromkeys(DEFAULT_MANIFEST)

{'dimensions': None,
 'statebits': None,
 'states': None,
 'dtypes': None,
 'color_palette': None,
 'graphics': None}

In [ ]:
d = {"floor": {"fixed_bits": [0,0,None,None],
                "allowed_states": []}
    }



In [ ]:


d

In [ ]:
current_dim2 = np.dstack([current]*4).reshape(80, 50, 2, 4)
current_dim2[8:12, 8:12, 1, :]

In [ ]:
state1 = np.array([True, True])
state2 = np.array([True, False])
state3 = np.array([False, True])
state4 = np.array([False, False])

states = np.vstack([state1, state2, state3, state4])
dim1 = np.concatenate([states]*50).reshape(50, *states.shape)
dim2 = np.concatenate([dim1]*80).reshape(80, 50, *states.shape)
dim2

In [ ]:
tile_map.tiles['graphic_type'][:]['name']



In [ ]:
for key in DEFAULT_GRAPHICS.keys():
    print(f"{key}: {DEFAULT_GRAPHICS[key] == tile_map._graphics_manifest[key]}")

In [ ]:
for key in tile_map._graphics_manifest['dtypes'].keys():
    print(f"{key}: {tile_map._graphics_manifest['dtypes'][key] == DEFAULT_GRAPHICS['dtypes'][key]}")

In [ ]:
DEFAULT_GRAPHICS['dtypes']

In [ ]:
tile_map._graphics_manifest['dtypes']

In [ ]:
area._align_center()
print(area.center)
area._align_corners()
print(area.top_left, area.bottom_right)

In [ ]:
import random
x,y =random.choice(np.argwhere(a))
int(x), int(y)

In [ ]:
def coordinate_overlap(dimension1: TileTuple, dimension2: TileTuple) -> bool:
    dim1_set1 = set(dimension1[0])
    dim1_set2 = set(dimension2[0])
    dim1_overlap = dim1_set1.is

    dim2_set1 = set(dimension1[1])
    dim2_set2 = set(dimension2[1])
    dim2_overlap = dim2_set1.isdisjoint(dim2_set2)
    
    return not (dim1_overlap or dim2_overlap)  

In [ ]:
a = set([1,2,3])
b = set([0,2])

b.issubset(a)

In [ ]:
a = np.full((5, 5), fill_value=False, dtype=bool)
a[mask_array] = True
a

In [ ]:
a &= mask_array
a

In [ ]:

dim1 = TileTuple((tiles[1][0].tolist()[1:3], tiles[1][0].tolist()[1:4]))
dim2 = TileTuple(([3,4], [3,4]))
coordinate_overlap(dim1, dim2)

In [ ]:
point1 = (1, 2)
point2 = (1, 3)


dim1 = ([point1[0]], [point1[1]])
dim2 = ([point2[0]], [point2[1]])

overlap(dim1, dim2)

In [ ]:
point1 = (0, 0)
point2 = (3, 4)
point3 = (4, 4)
point4 = (6, 8)


a = (range(point1[0], point2[0]), range(point1[1], point2[1]))
b = (range(point3[0], point4[0]), range(point3[1], point4[1]))

overlap(a, b)

In [ ]:
point1 = (1, 1)
point2 = (3, 3)
point3 = (4, 4)
point4 = (6, 6)

a = np.array([[1, 3], [1, 3]])  
b = np.array([[4, 6], [4, 6]])

overlap(a, b)

In [ ]:
point1 = (1, 1)
point2 = (3, 3)
point3 = (1, 1)
point4 = (3, 3)

a = np.array([[1, 3], [1, 3]])  
b = np.array([[1, 3], [1, 3]])

overlap(a, b)

In [ ]:
point1 = (1, 1)
point2 = (3, 5)
point3 = (4, 4)
point4 = (5, 5)

a = np.array([np.arange(1,4), np.arange(1,6)])  
b = np.array([np.arange(3, 6), np.arange(3, 6)])

overlap(a, b)

In [ ]:
from core_components.generators.random import GenericDungeonGenerator, CIRCULAR_ROOM_TEMPLATE
from core_components.tilemaps.base import MapCoords
import matplotlib.pyplot as plt
import numpy as np
dungeon_gen = GenericDungeonGenerator((50,50))
room_template = CIRCULAR_ROOM_TEMPLATE
center = MapCoords(15, 15)
size = (10, 8)

# Act
instance = dungeon_gen.generate()

In [ ]:
from core_components.tilemaps.base import MapCoords
a = MapCoords(14, 15)
b = MapCoords(14, 15)
a == b

In [ ]:

plt.imshow(instance.tiles['transparent'])

In [ ]:
dungeon_gen.corridors[0]

In [ ]:
plt.imshow(dungeon_gen.spawn_room(CIRCULAR_ROOM_TEMPLATE, center, size).inner_area)

In [ ]:
mask = np.full((width, height), False)
mask = np.s_[2:5, 2:5]

tile_map._tiles['type'][mask] = tile_map.resources['tiles']['floor']
tile_map._tiles[mask]['type']

In [ ]:
def istype(a: np.ndarray, dtype: np.dtype) -> bool:
    """Helper function to check if a numpy array is of a specific dtype"""
    if a.dtype.metadata is None or dtype.metadata is None:
        return False
    return a.dtype.metadata['__name__'] == dtype.metadata['__name__']

ascii_default = np.array((ord(' '), (255, 255, 255), (0, 0, 0)), dtype=ascii_graphic)

istype(ascii_default, ascii_graphic)  # True

In [ ]:
def run_tests(test_list):
    count = 0
    for test in test_list:
        try:
            test()
            count += 1
        except Exception as e:
            print(f"Exception: {test.__name__}: {e}")
            continue
    print(f"{count} of {len(test_list)} tests ran successfully.")

In [ ]:
from test.test_component_actions_system_dispatcher import *
tests = [test_dispatcher_ev_gamestart,
         test_dispatcher_ev_gameover,
         test_dispatcher_ev_keydown]

run_tests(tests)
# --- IGNORE ---

In [ ]:
from test.test_component_actions_dispatchers import *
tests = [test_dispatcher_ev_quit,
         test_dispatcher_initialization]

run_tests(tests)
# --- IGNORE ---

In [ ]:
from test.test_game_ai import *
test_game_over_event_handling()
test_game_ai_initialization()
test_game_ai_no_events()
test_game_ai_multiple_events()
test_game_ai_event_clearing()



In [ ]:
from test.test_game_map import *
test_get_map_coords()
test_in_bounds()
test_is_traversable_transparent_visible_explored()
test_out_of_bounds_checks()
test_set_visible_and_reset()
test_update_explored()
test_width_height()
test_tiles_initialization()


In [ ]:
from test.test_roster import *
test_roster_spawn()
test_roster_entity_blocked_locations()
test_roster_entity_collision()
test_roster_live_ai_actors()
test_roster_live_actors()
test_roster_get_entity_at_location()
